In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_train=pd.read_csv('C:/Users/vaiko/Desktop/Rishi_Raman/Analytics_vid_ref/Janata_Hack_Banking/Data_files/train_fNxu4vz.csv')
df_test=pd.read_csv('C:/Users/vaiko/Desktop/Rishi_Raman/Analytics_vid_ref/Janata_Hack_Banking/Data_files/test_fjtUOL8.csv')
df_sub=pd.read_csv('C:/Users/vaiko/Desktop/Rishi_Raman/Analytics_vid_ref/Janata_Hack_Banking/Data_files/sample_submission_HSqiq1Q.csv')
print('train shape',df_train.shape)
print('test shape',df_test.shape)

train shape (164309, 14)
test shape (109541, 13)


In [5]:
df_train.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
3,10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1


In [167]:
def basic_work(data):
    new_data=data
    new_data['Loan_Amount_Requested']=new_data['Loan_Amount_Requested'].replace({',':''},regex=True).astype(int)
    return new_data



def feature_engineering(data):
    new_data=data
    new_data['Length_Employed']=new_data['Length_Employed'].fillna(data['Length_Employed'].mode()[0])
    new_data['Home_Owner']=new_data['Home_Owner'].fillna(data['Home_Owner'].mode()[0])
    new_data['Annual_Income']=new_data['Annual_Income'].fillna(data['Annual_Income'].median())
    new_data['Months_Since_Deliquency']=new_data['Months_Since_Deliquency'].fillna(data['Months_Since_Deliquency'].mode()[0])
#     new_data=pd.get_dummies(new_data,columns=['Income_Verified','Gender'],drop_first=True)
    x=pd.DataFrame(new_data['Length_Employed'].value_counts(normalize=True).round(3))
    x['val']=x.index
    x.columns=['rank_val_len_emplo','Length_Employed']
    x.reset_index(drop=True,inplace=True)
    new_data=pd.merge(left=new_data,right=x,on='Length_Employed',how='left')
    
    new_data['sect_amount']=pd.qcut(new_data['Loan_Amount_Requested'],q=4)
    from sklearn.preprocessing import LabelEncoder
    lb6=LabelEncoder()
    new_data['sect_amount']=lb6.fit_transform(new_data['sect_amount'])
    
    new_data['sect_annual_in']=pd.qcut(new_data['Annual_Income'],q=4)
    from sklearn.preprocessing import LabelEncoder
    lb7=LabelEncoder()
    new_data['sect_annual_in']=lb7.fit_transform(new_data['sect_amount'])
    
    
    
    return new_data
    
def train_test_encode(train,test):   
    from sklearn.preprocessing import LabelEncoder
    lb1=LabelEncoder()
    lb1.fit(train['Length_Employed'])
    train['Length_Employed']=lb1.transform(train['Length_Employed'])
    test['Length_Employed']=lb1.fit_transform(test['Length_Employed'])
    
    lb2=LabelEncoder()
    lb2.fit(train['Home_Owner'])
    train['Home_Owner']=lb2.transform(train['Home_Owner'])
    test['Home_Owner']=lb2.fit_transform(test['Home_Owner'])
    
    lb3=LabelEncoder()
    lb3.fit(train['Purpose_Of_Loan'])
    train['Purpose_Of_Loan']=lb3.transform(train['Purpose_Of_Loan'])
    test['Purpose_Of_Loan']=lb3.fit_transform(test['Purpose_Of_Loan'])
    
    lb4=LabelEncoder()
    lb4.fit(train['Income_Verified'])
    train['Income_Verified']=lb4.transform(train['Income_Verified'])
    test['Income_Verified']=lb4.fit_transform(test['Income_Verified'])
    
    lb5=LabelEncoder()
    lb5.fit(train['Gender'])
    train['Gender']=lb5.transform(train['Gender'])
    test['Gender']=lb5.fit_transform(test['Gender'])

In [168]:
df_train=basic_work(df_train)
df_test=basic_work(df_test)
df_train=feature_engineering(df_train)
df_test=feature_engineering(df_test)
train_test_encode(df_train,df_test)

In [170]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

X=df_train.drop(columns=['Loan_ID','Interest_Rate'])
y=df_train["Interest_Rate"]



In [171]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [172]:
from catboost import CatBoostClassifier
cbc=CatBoostClassifier()

In [173]:
cbc.fit(X_train,y_train)

0:	learn: 1.0927886	total: 41ms	remaining: 40.9s
1:	learn: 1.0873233	total: 79.2ms	remaining: 39.5s
2:	learn: 1.0821160	total: 118ms	remaining: 39.2s
3:	learn: 1.0772336	total: 155ms	remaining: 38.7s
4:	learn: 1.0726255	total: 194ms	remaining: 38.7s
5:	learn: 1.0682463	total: 232ms	remaining: 38.4s
6:	learn: 1.0641029	total: 277ms	remaining: 39.3s
7:	learn: 1.0601324	total: 317ms	remaining: 39.3s
8:	learn: 1.0562813	total: 353ms	remaining: 38.9s
9:	learn: 1.0525687	total: 390ms	remaining: 38.6s
10:	learn: 1.0489604	total: 427ms	remaining: 38.3s
11:	learn: 1.0455870	total: 472ms	remaining: 38.8s
12:	learn: 1.0423898	total: 510ms	remaining: 38.7s
13:	learn: 1.0393985	total: 548ms	remaining: 38.6s
14:	learn: 1.0366282	total: 586ms	remaining: 38.5s
15:	learn: 1.0337659	total: 629ms	remaining: 38.7s
16:	learn: 1.0310229	total: 670ms	remaining: 38.7s
17:	learn: 1.0283804	total: 709ms	remaining: 38.7s
18:	learn: 1.0258050	total: 747ms	remaining: 38.6s
19:	learn: 1.0234645	total: 784ms	remaini

166:	learn: 0.9390253	total: 6.5s	remaining: 32.4s
167:	learn: 0.9389177	total: 6.53s	remaining: 32.4s
168:	learn: 0.9387854	total: 6.57s	remaining: 32.3s
169:	learn: 0.9386827	total: 6.6s	remaining: 32.2s
170:	learn: 0.9385508	total: 6.64s	remaining: 32.2s
171:	learn: 0.9384549	total: 6.68s	remaining: 32.2s
172:	learn: 0.9383312	total: 6.71s	remaining: 32.1s
173:	learn: 0.9381842	total: 6.75s	remaining: 32s
174:	learn: 0.9380376	total: 6.79s	remaining: 32s
175:	learn: 0.9379349	total: 6.82s	remaining: 31.9s
176:	learn: 0.9378138	total: 6.86s	remaining: 31.9s
177:	learn: 0.9376902	total: 6.89s	remaining: 31.8s
178:	learn: 0.9375632	total: 6.93s	remaining: 31.8s
179:	learn: 0.9374716	total: 6.96s	remaining: 31.7s
180:	learn: 0.9373257	total: 7s	remaining: 31.7s
181:	learn: 0.9371879	total: 7.03s	remaining: 31.6s
182:	learn: 0.9370728	total: 7.07s	remaining: 31.6s
183:	learn: 0.9369767	total: 7.11s	remaining: 31.5s
184:	learn: 0.9368512	total: 7.15s	remaining: 31.5s
185:	learn: 0.9367116

326:	learn: 0.9267567	total: 12.3s	remaining: 25.4s
327:	learn: 0.9267209	total: 12.4s	remaining: 25.4s
328:	learn: 0.9266855	total: 12.4s	remaining: 25.3s
329:	learn: 0.9266354	total: 12.4s	remaining: 25.3s
330:	learn: 0.9265856	total: 12.5s	remaining: 25.2s
331:	learn: 0.9265313	total: 12.5s	remaining: 25.2s
332:	learn: 0.9264813	total: 12.6s	remaining: 25.2s
333:	learn: 0.9264483	total: 12.6s	remaining: 25.1s
334:	learn: 0.9264031	total: 12.6s	remaining: 25.1s
335:	learn: 0.9263305	total: 12.7s	remaining: 25s
336:	learn: 0.9262675	total: 12.7s	remaining: 25s
337:	learn: 0.9262253	total: 12.7s	remaining: 25s
338:	learn: 0.9261794	total: 12.8s	remaining: 24.9s
339:	learn: 0.9261455	total: 12.8s	remaining: 24.9s
340:	learn: 0.9261192	total: 12.8s	remaining: 24.8s
341:	learn: 0.9260662	total: 12.9s	remaining: 24.8s
342:	learn: 0.9260143	total: 12.9s	remaining: 24.8s
343:	learn: 0.9259560	total: 13s	remaining: 24.7s
344:	learn: 0.9259068	total: 13s	remaining: 24.7s
345:	learn: 0.9258640	

489:	learn: 0.9190024	total: 18.5s	remaining: 19.3s
490:	learn: 0.9189512	total: 18.6s	remaining: 19.2s
491:	learn: 0.9189129	total: 18.6s	remaining: 19.2s
492:	learn: 0.9188847	total: 18.6s	remaining: 19.1s
493:	learn: 0.9188335	total: 18.7s	remaining: 19.1s
494:	learn: 0.9187767	total: 18.7s	remaining: 19.1s
495:	learn: 0.9187364	total: 18.7s	remaining: 19s
496:	learn: 0.9187040	total: 18.8s	remaining: 19s
497:	learn: 0.9186733	total: 18.8s	remaining: 18.9s
498:	learn: 0.9186381	total: 18.8s	remaining: 18.9s
499:	learn: 0.9185874	total: 18.9s	remaining: 18.9s
500:	learn: 0.9185318	total: 18.9s	remaining: 18.8s
501:	learn: 0.9184861	total: 18.9s	remaining: 18.8s
502:	learn: 0.9184525	total: 19s	remaining: 18.7s
503:	learn: 0.9184240	total: 19s	remaining: 18.7s
504:	learn: 0.9183719	total: 19s	remaining: 18.7s
505:	learn: 0.9183440	total: 19.1s	remaining: 18.6s
506:	learn: 0.9182921	total: 19.1s	remaining: 18.6s
507:	learn: 0.9182502	total: 19.2s	remaining: 18.6s
508:	learn: 0.9182196	

651:	learn: 0.9132690	total: 24.3s	remaining: 13s
652:	learn: 0.9132457	total: 24.4s	remaining: 13s
653:	learn: 0.9132265	total: 24.4s	remaining: 12.9s
654:	learn: 0.9131897	total: 24.4s	remaining: 12.9s
655:	learn: 0.9131581	total: 24.5s	remaining: 12.8s
656:	learn: 0.9131308	total: 24.5s	remaining: 12.8s
657:	learn: 0.9130969	total: 24.6s	remaining: 12.8s
658:	learn: 0.9130590	total: 24.6s	remaining: 12.7s
659:	learn: 0.9130334	total: 24.6s	remaining: 12.7s
660:	learn: 0.9130057	total: 24.7s	remaining: 12.6s
661:	learn: 0.9129863	total: 24.7s	remaining: 12.6s
662:	learn: 0.9129634	total: 24.7s	remaining: 12.6s
663:	learn: 0.9129321	total: 24.8s	remaining: 12.5s
664:	learn: 0.9129020	total: 24.8s	remaining: 12.5s
665:	learn: 0.9128670	total: 24.8s	remaining: 12.5s
666:	learn: 0.9128355	total: 24.9s	remaining: 12.4s
667:	learn: 0.9128155	total: 24.9s	remaining: 12.4s
668:	learn: 0.9127857	total: 24.9s	remaining: 12.3s
669:	learn: 0.9127630	total: 25s	remaining: 12.3s
670:	learn: 0.9127

811:	learn: 0.9089642	total: 30.3s	remaining: 7.01s
812:	learn: 0.9089447	total: 30.3s	remaining: 6.97s
813:	learn: 0.9089190	total: 30.3s	remaining: 6.93s
814:	learn: 0.9089017	total: 30.4s	remaining: 6.89s
815:	learn: 0.9088763	total: 30.4s	remaining: 6.86s
816:	learn: 0.9088545	total: 30.4s	remaining: 6.82s
817:	learn: 0.9088178	total: 30.5s	remaining: 6.78s
818:	learn: 0.9088000	total: 30.5s	remaining: 6.74s
819:	learn: 0.9087725	total: 30.5s	remaining: 6.71s
820:	learn: 0.9087425	total: 30.6s	remaining: 6.67s
821:	learn: 0.9086996	total: 30.6s	remaining: 6.63s
822:	learn: 0.9086680	total: 30.7s	remaining: 6.59s
823:	learn: 0.9086353	total: 30.7s	remaining: 6.56s
824:	learn: 0.9086043	total: 30.7s	remaining: 6.52s
825:	learn: 0.9085720	total: 30.8s	remaining: 6.48s
826:	learn: 0.9085593	total: 30.8s	remaining: 6.44s
827:	learn: 0.9085393	total: 30.8s	remaining: 6.41s
828:	learn: 0.9085192	total: 30.9s	remaining: 6.37s
829:	learn: 0.9084884	total: 30.9s	remaining: 6.33s
830:	learn: 

974:	learn: 0.9051489	total: 36.2s	remaining: 927ms
975:	learn: 0.9051155	total: 36.2s	remaining: 890ms
976:	learn: 0.9050836	total: 36.2s	remaining: 853ms
977:	learn: 0.9050619	total: 36.3s	remaining: 816ms
978:	learn: 0.9050365	total: 36.3s	remaining: 778ms
979:	learn: 0.9049986	total: 36.3s	remaining: 741ms
980:	learn: 0.9049764	total: 36.4s	remaining: 704ms
981:	learn: 0.9049581	total: 36.4s	remaining: 667ms
982:	learn: 0.9049242	total: 36.4s	remaining: 630ms
983:	learn: 0.9049032	total: 36.5s	remaining: 593ms
984:	learn: 0.9048795	total: 36.5s	remaining: 556ms
985:	learn: 0.9048484	total: 36.5s	remaining: 519ms
986:	learn: 0.9048206	total: 36.6s	remaining: 482ms
987:	learn: 0.9048006	total: 36.6s	remaining: 445ms
988:	learn: 0.9047885	total: 36.7s	remaining: 408ms
989:	learn: 0.9047586	total: 36.7s	remaining: 371ms
990:	learn: 0.9047279	total: 36.7s	remaining: 334ms
991:	learn: 0.9047076	total: 36.8s	remaining: 296ms
992:	learn: 0.9046840	total: 36.8s	remaining: 259ms
993:	learn: 

In [174]:
pred=cbc.predict(X_test)

In [175]:
from sklearn.metrics import classification_report,accuracy_score

In [176]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           1       0.54      0.26      0.35     10017
           2       0.51      0.64      0.57     21255
           3       0.59      0.58      0.59     18021

    accuracy                           0.54     49293
   macro avg       0.55      0.49      0.50     49293
weighted avg       0.54      0.54      0.53     49293



In [177]:
print(accuracy_score(y_test,pred))

0.5399346763232102


In [178]:
x=pd.DataFrame(X_train.columns,cbc.feature_importances_)

In [179]:
x.sort_index(ascending=False)

,0
13.783655,Loan_Amount_Requested
12.363683,Inquiries_Last_6Mo
11.511321,Months_Since_Deliquency
11.034794,Purpose_Of_Loan
10.549473,Debt_To_Income
9.899525,Total_Accounts
9.238149,Annual_Income
9.229176,Income_Verified
3.220699,Number_Open_Accounts
2.472932,sect_annual_in


In [180]:
from catboost import CatBoostClassifier
cbc2=CatBoostClassifier()

In [181]:
cbc2.fit(X,y)

0:	learn: 1.0927673	total: 57.9ms	remaining: 57.9s
1:	learn: 1.0872643	total: 112ms	remaining: 55.8s
2:	learn: 1.0820386	total: 168ms	remaining: 55.9s
3:	learn: 1.0771273	total: 222ms	remaining: 55.4s
4:	learn: 1.0724526	total: 285ms	remaining: 56.7s
5:	learn: 1.0680581	total: 343ms	remaining: 56.8s
6:	learn: 1.0639640	total: 400ms	remaining: 56.7s
7:	learn: 1.0600118	total: 457ms	remaining: 56.7s
8:	learn: 1.0560881	total: 519ms	remaining: 57.2s
9:	learn: 1.0523539	total: 578ms	remaining: 57.2s
10:	learn: 1.0487620	total: 631ms	remaining: 56.7s
11:	learn: 1.0454604	total: 686ms	remaining: 56.4s
12:	learn: 1.0422117	total: 746ms	remaining: 56.6s
13:	learn: 1.0391385	total: 802ms	remaining: 56.5s
14:	learn: 1.0363493	total: 856ms	remaining: 56.2s
15:	learn: 1.0334853	total: 914ms	remaining: 56.2s
16:	learn: 1.0306162	total: 975ms	remaining: 56.4s
17:	learn: 1.0280299	total: 1.03s	remaining: 56.3s
18:	learn: 1.0254641	total: 1.09s	remaining: 56.2s
19:	learn: 1.0231036	total: 1.15s	remain

161:	learn: 0.9394998	total: 9.61s	remaining: 49.7s
162:	learn: 0.9393687	total: 9.66s	remaining: 49.6s
163:	learn: 0.9392119	total: 9.72s	remaining: 49.5s
164:	learn: 0.9390906	total: 9.77s	remaining: 49.5s
165:	learn: 0.9389514	total: 9.83s	remaining: 49.4s
166:	learn: 0.9388374	total: 9.88s	remaining: 49.3s
167:	learn: 0.9387121	total: 9.94s	remaining: 49.2s
168:	learn: 0.9385947	total: 10s	remaining: 49.2s
169:	learn: 0.9384740	total: 10.1s	remaining: 49.1s
170:	learn: 0.9383520	total: 10.1s	remaining: 49s
171:	learn: 0.9382602	total: 10.2s	remaining: 48.9s
172:	learn: 0.9381207	total: 10.2s	remaining: 48.9s
173:	learn: 0.9379940	total: 10.3s	remaining: 48.8s
174:	learn: 0.9378642	total: 10.3s	remaining: 48.7s
175:	learn: 0.9377632	total: 10.4s	remaining: 48.6s
176:	learn: 0.9376598	total: 10.5s	remaining: 48.6s
177:	learn: 0.9375672	total: 10.5s	remaining: 48.6s
178:	learn: 0.9374581	total: 10.6s	remaining: 48.5s
179:	learn: 0.9373447	total: 10.7s	remaining: 48.5s
180:	learn: 0.93

324:	learn: 0.9276597	total: 19.3s	remaining: 40.1s
325:	learn: 0.9276152	total: 19.4s	remaining: 40s
326:	learn: 0.9275701	total: 19.4s	remaining: 40s
327:	learn: 0.9274899	total: 19.5s	remaining: 39.9s
328:	learn: 0.9274324	total: 19.5s	remaining: 39.8s
329:	learn: 0.9274039	total: 19.6s	remaining: 39.8s
330:	learn: 0.9273847	total: 19.6s	remaining: 39.7s
331:	learn: 0.9273203	total: 19.7s	remaining: 39.6s
332:	learn: 0.9272892	total: 19.7s	remaining: 39.6s
333:	learn: 0.9272259	total: 19.8s	remaining: 39.5s
334:	learn: 0.9271934	total: 19.9s	remaining: 39.4s
335:	learn: 0.9271422	total: 19.9s	remaining: 39.4s
336:	learn: 0.9270644	total: 20s	remaining: 39.3s
337:	learn: 0.9270255	total: 20s	remaining: 39.2s
338:	learn: 0.9269833	total: 20.1s	remaining: 39.1s
339:	learn: 0.9269499	total: 20.1s	remaining: 39.1s
340:	learn: 0.9269153	total: 20.2s	remaining: 39s
341:	learn: 0.9268684	total: 20.2s	remaining: 38.9s
342:	learn: 0.9268196	total: 20.3s	remaining: 38.8s
343:	learn: 0.9267657	

483:	learn: 0.9209891	total: 28.6s	remaining: 30.5s
484:	learn: 0.9209654	total: 28.6s	remaining: 30.4s
485:	learn: 0.9209320	total: 28.7s	remaining: 30.3s
486:	learn: 0.9208933	total: 28.7s	remaining: 30.3s
487:	learn: 0.9208580	total: 28.8s	remaining: 30.2s
488:	learn: 0.9208171	total: 28.9s	remaining: 30.2s
489:	learn: 0.9208012	total: 29s	remaining: 30.1s
490:	learn: 0.9207434	total: 29s	remaining: 30.1s
491:	learn: 0.9206991	total: 29.1s	remaining: 30.1s
492:	learn: 0.9206638	total: 29.2s	remaining: 30s
493:	learn: 0.9206327	total: 29.3s	remaining: 30s
494:	learn: 0.9205908	total: 29.4s	remaining: 30s
495:	learn: 0.9205562	total: 29.5s	remaining: 30s
496:	learn: 0.9205007	total: 29.5s	remaining: 29.9s
497:	learn: 0.9204740	total: 29.6s	remaining: 29.8s
498:	learn: 0.9204383	total: 29.7s	remaining: 29.8s
499:	learn: 0.9204137	total: 29.7s	remaining: 29.7s
500:	learn: 0.9203862	total: 29.8s	remaining: 29.7s
501:	learn: 0.9203466	total: 29.9s	remaining: 29.6s
502:	learn: 0.9203115	to

643:	learn: 0.9159699	total: 37.6s	remaining: 20.8s
644:	learn: 0.9159421	total: 37.7s	remaining: 20.7s
645:	learn: 0.9159240	total: 37.7s	remaining: 20.7s
646:	learn: 0.9158961	total: 37.8s	remaining: 20.6s
647:	learn: 0.9158687	total: 37.8s	remaining: 20.5s
648:	learn: 0.9158318	total: 37.9s	remaining: 20.5s
649:	learn: 0.9158074	total: 37.9s	remaining: 20.4s
650:	learn: 0.9157867	total: 38s	remaining: 20.4s
651:	learn: 0.9157586	total: 38s	remaining: 20.3s
652:	learn: 0.9157344	total: 38.1s	remaining: 20.2s
653:	learn: 0.9157038	total: 38.1s	remaining: 20.2s
654:	learn: 0.9156778	total: 38.2s	remaining: 20.1s
655:	learn: 0.9156575	total: 38.3s	remaining: 20.1s
656:	learn: 0.9156392	total: 38.3s	remaining: 20s
657:	learn: 0.9155998	total: 38.4s	remaining: 19.9s
658:	learn: 0.9155705	total: 38.4s	remaining: 19.9s
659:	learn: 0.9155521	total: 38.5s	remaining: 19.8s
660:	learn: 0.9155268	total: 38.5s	remaining: 19.8s
661:	learn: 0.9155086	total: 38.6s	remaining: 19.7s
662:	learn: 0.9154

803:	learn: 0.9123573	total: 47s	remaining: 11.5s
804:	learn: 0.9123298	total: 47.1s	remaining: 11.4s
805:	learn: 0.9122911	total: 47.1s	remaining: 11.3s
806:	learn: 0.9122608	total: 47.2s	remaining: 11.3s
807:	learn: 0.9122420	total: 47.3s	remaining: 11.2s
808:	learn: 0.9122251	total: 47.3s	remaining: 11.2s
809:	learn: 0.9122061	total: 47.4s	remaining: 11.1s
810:	learn: 0.9121861	total: 47.4s	remaining: 11.1s
811:	learn: 0.9121602	total: 47.5s	remaining: 11s
812:	learn: 0.9121424	total: 47.5s	remaining: 10.9s
813:	learn: 0.9121221	total: 47.6s	remaining: 10.9s
814:	learn: 0.9121022	total: 47.7s	remaining: 10.8s
815:	learn: 0.9120867	total: 47.7s	remaining: 10.8s
816:	learn: 0.9120701	total: 47.8s	remaining: 10.7s
817:	learn: 0.9120384	total: 47.8s	remaining: 10.6s
818:	learn: 0.9120271	total: 47.9s	remaining: 10.6s
819:	learn: 0.9120080	total: 48s	remaining: 10.5s
820:	learn: 0.9119913	total: 48s	remaining: 10.5s
821:	learn: 0.9119747	total: 48.1s	remaining: 10.4s
822:	learn: 0.911956

963:	learn: 0.9091858	total: 56.6s	remaining: 2.11s
964:	learn: 0.9091654	total: 56.7s	remaining: 2.05s
965:	learn: 0.9091455	total: 56.7s	remaining: 2s
966:	learn: 0.9091211	total: 56.8s	remaining: 1.94s
967:	learn: 0.9091069	total: 56.8s	remaining: 1.88s
968:	learn: 0.9090903	total: 56.9s	remaining: 1.82s
969:	learn: 0.9090710	total: 57s	remaining: 1.76s
970:	learn: 0.9090524	total: 57s	remaining: 1.7s
971:	learn: 0.9090357	total: 57.1s	remaining: 1.64s
972:	learn: 0.9090173	total: 57.1s	remaining: 1.58s
973:	learn: 0.9090037	total: 57.2s	remaining: 1.53s
974:	learn: 0.9089887	total: 57.3s	remaining: 1.47s
975:	learn: 0.9089745	total: 57.3s	remaining: 1.41s
976:	learn: 0.9089591	total: 57.4s	remaining: 1.35s
977:	learn: 0.9089369	total: 57.4s	remaining: 1.29s
978:	learn: 0.9089145	total: 57.5s	remaining: 1.23s
979:	learn: 0.9088941	total: 57.5s	remaining: 1.17s
980:	learn: 0.9088813	total: 57.6s	remaining: 1.11s
981:	learn: 0.9088600	total: 57.7s	remaining: 1.06s
982:	learn: 0.908840

In [182]:
df_test=df_test.drop(columns=['Loan_ID'])

In [183]:
pred_ou=cbc2.predict(df_test)

In [184]:
df_sub['Interest_Rate']=pred_ou

In [185]:
df_sub.to_csv('output1.csv',index=False)